# Analysis of Real Estate in Malta

In [ ]:
loan_years = 25
loan_rate = 0.03

downpayment_rate = 0.25

vacancy_rate = 0.05
repair_rate = 0.05
management_rate = 0.1

## 1.1 - Load Dataset

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats

dataset = pd.read_csv('../dataset.csv')

In [ ]:
dataset.loc[(dataset['Is_Sale'] == True), 'Case'] = 1
dataset.loc[(dataset['Is_Sale'] != True), 'Case'] = 2

dataset

In [ ]:
res_sale = dataset[dataset['Case'] == 1]
res_sale = res_sale[(np.abs(stats.zscore(res_sale['Price'])) < 3)]

res_rent = dataset[dataset['Case'] == 2]
res_rent = res_rent[(np.abs(stats.zscore(res_rent['Price'])) < 3)]

## 1.2 - Missing Values

In [ ]:
import plotly.express as px

chart_data = dataset.isna().sum()
fig = px.pie(
    values=chart_data.values,
    names=chart_data.index,
    title="Missing Values"
)

fig.show()

In [ ]:
dataset[dataset.columns[7:]] = dataset[dataset.columns[7:]].fillna('Unclassified')

## 1.3 - General Visuals

In [ ]:
chart_data = dataset['Case'].value_counts()
fig = px.pie(
    values=chart_data.values,
    names=['Sale', 'Rent'],
    title="Dataset by Case"
)

fig.show()

In [ ]:
chart_data = dataset['Source'].value_counts()
fig = px.pie(
    values=chart_data.values,
    names=chart_data.index,
    title="Dataset by Source"
)

fig.show()

In [ ]:
chart_data = dataset['Province'].value_counts()
fig = px.pie(
    values=chart_data.values,
    names=chart_data.index,
    title="Dataset by Province"
)

fig.show()

In [ ]:
fig = px.scatter_mapbox(
    dataset, lat=dataset['Latitude'], lon=dataset['Longitude'], color_discrete_sequence=[dataset['Case']],
    hover_name="Reference", hover_data=["Case", "Price"]
)

fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## 1.4 - Property Type Distribution

In [ ]:
fig = px.box(res_sale, x="Type", y="Price", title="Residential Sale - Price Vs Property Type")
fig.show()

In [ ]:
fig = px.box(res_rent, x="Type", y="Price", title="Residential Rent - Price Vs Property Type")
fig.show()

## 1.5 - Correlation between Sale and Rent

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

valid_cols = [
    'Rooms', 'Bedrooms', 'Bathrooms', 
    'TotalSqm', 'IntArea', 'ExtArea', 'Price'
]

corr_matrix = res_sale[valid_cols].corr(numeric_only=True)
sn.heatmap(corr_matrix, annot=True, cmap='BrBG')
plt.show()

In [ ]:
corr_matrix = res_rent[valid_cols].corr(numeric_only=True)
sn.heatmap(corr_matrix, annot=True, cmap='BrBG')
plt.show()

## 1.6 - Aggregated Data

In [ ]:
group_by_cols = [
    'Type', 'Town', 'Bedrooms'
]

agg_fun = {
    'Reference': 'count',
    'Bathrooms': 'median',
    'IntArea': 'median',
    'ExtArea': 'median',
    'Price': ['min', 'max', 'mean']
}

In [ ]:
res_sale_agg = res_sale.groupby(group_by_cols).agg(agg_fun).round(2)
res_rent_agg = res_rent.groupby(group_by_cols).agg(agg_fun).round(2)

res_sale_agg.columns = ['_'.join(col) for col in res_sale_agg.columns.values]
res_rent_agg.columns = ['_'.join(col) for col in res_rent_agg.columns.values]

In [ ]:
res_agg = res_sale_agg.merge(res_rent_agg, on=group_by_cols)
res_agg_flt = res_agg[(res_agg['Reference_count_x'] > 1) & (res_agg['Reference_count_y'] > 1)].round(2).reset_index()

res_agg_flt

In [ ]:
residential = pd.DataFrame()

residential[group_by_cols] = res_agg_flt[group_by_cols]

residential[[
    'Sale_Count', 'Rent_Count', 
    'Sale_Median_Bathrooms', 'Rent_Median_Bathrooms',
    'Sale_Median_IntArea', 'Rent_Median_IntArea',
    'Sale_Median_ExtArea', 'Rent_Median_ExtArea', 
    'Sale_Min_Price', 'Sale_Max_Price', 'Sale_Mean_Price',
    'Rent_Min_Price', 'Rent_Max_Price', 'Rent_Mean_Price'
]] = res_agg_flt[[
    'Reference_count_x', 'Reference_count_y',
    'Bathrooms_median_x', 'Bathrooms_median_y',
    'IntArea_median_x', 'IntArea_median_y',
    'ExtArea_median_x', 'ExtArea_median_y',
    'Price_min_x', 'Price_max_x', 'Price_mean_x',
    'Price_min_y', 'Price_max_y', 'Price_mean_y'
]]

residential['Downpayment'] = residential['Sale_Mean_Price'] * downpayment_rate

residential['Repayment_Year'] = (residential['Downpayment'] * loan_rate) / (1 - (1 + loan_rate) ** - loan_years)
residential['Repayment_Monthly'] = residential['Repayment_Year'] / 12

residential['Vacancy_Monthly'] = residential['Rent_Mean_Price'] * vacancy_rate
residential['Repairs_Monthly'] = residential['Rent_Mean_Price'] * repair_rate
residential['Management_Monthly'] = residential['Rent_Mean_Price'] * management_rate

residential['Net_Rent_Year'] = (residential['Rent_Mean_Price'] - (
    residential['Repayment_Monthly'] + residential['Vacancy_Monthly'] +
    residential['Repairs_Monthly'] + residential['Management_Monthly']
)) * 12

residential['Net_Rent_Monthly'] = residential['Net_Rent_Year'] / 12

residential['CoC_Return'] = residential['Net_Rent_Year'] / residential['Downpayment']

residential = residential.round(2)
residential

## 1.7 - Rank Properties

In [ ]:
res_coc = residential.sort_values('CoC_Return', ascending=False)
res_coc = res_coc[res_coc['Type'] != 'Garage (Residential)']

res_coc.to_csv('../output.csv', index=False)

In [ ]:
top_10 = res_coc.head(10)
top_10